## 第 10 章 分析句子的意思

我们已经看到利用计算机的能力来处理大规模文本是多么有用。现在我们已经有了分析器和基于特征的文法，我们能否做一些类似分析句子的意思这样有用的事情？

本章的目的是要回答下列问题： 
1. 我们如何能表示自然语言的意思，使计算机能够处理这些表示？ 
2. 我们怎样才能将意思表示与无限的句子集合关联？ 
3. 我们怎样才能使用连接意思表示与句子的程序来存储知识？ 

一路上，我们将学习一些逻辑语义领域的形式化技术，看看如何用它们来查询存储了世间真知的数据库。


### 10.1 自然语言理解

#### 查询数据库
`SQL`（`Structured Query Language`，结构化查询语言）是为在关系数据库中检索和管理数据而设计的语言。

我们怎样才能使用英语得到与我们在查询系统中输入得到的相同的效果呢？第9 章中描述的基于特征的文法形式可以很容易地从英语翻译到`SQL`。文法 `sql0.fcfg`说明如何将句子意思表示与句子分析串联组装。每个短语结构规则为特征`SEM` 构建值作补充。你可以看到这些补充非常简单；在每一种情况下，我们对分割的子成分用字符串连接操作+来组成父成分的值。

In [1]:
import nltk
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [9]:
# 这使我们能够分析SQL 查询：
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'
trees = cp.parse(query.split())
# answer = trees[0].node['sem']
answer = list(trees)[0].label()['SEM']
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE   Country="china"


In [11]:
# 最后，我们在数据库city.db上执行查询，检索出一些结果：
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0])

canton
chungking
dairen
harbin
kowloon
mukden
peking
shanghai
sian
tientsin


总结一下，我们已经定义了一个任务：计算机对自然语言查询做出反应，返回有用的数据。我们通过将英语的一个小的子集翻译成`SQL` 来实现这个任务们可以说，我们的 NLTK 代码已经“理解”`SQL`，只要 Python能够对数据库执行 `SQL` 查询，通过扩展，它也“理解” 如“What cities are located in China”这样的查询。

文法 `sql0.fcfg`，连同NLTK 的`Earley` 分析器是实现从英语翻译到 `SQL` 的工具。这个文法够用吗？你已经看到整个句子的 `SQL` 翻译是由句子成分的翻译建立起来的。然而，这些成分的意思表示似乎没有很多的合理性。

在下面的章节中，我们将探讨将自然语言的句子翻 译成逻辑而不是如SQL 这样的可执行查询语言的方法。逻辑形式的一个优势是它们更抽象， 因此更通用。一旦我们翻译成了逻辑，只要我们想要，就可以再翻译成其他各种特殊用途的 语言。事实上，大多说通过自然语言查询数据库的重要的尝试都是使用这种方法。

#### 自然语言、语义和逻辑

我们引进语义中的两个基本概念。第一个是在某些情况下，陈述句非真即假。第二个是名词短语和专有名词的定义指的是世界上的东西。

从广义上讲，自然语言语义表示的基于逻辑的方法关注那些指导我们判断自然语言的一 致性和不一致性的方面。设计一种逻辑语言的句法是为了使这些特征形式更明确。结果是如一致性这样的确定性属性往往可以简化成符号操作，也就是说，一种可以被计算机实施的任务。为了实现这种方法，我们首先要开发一种表示某种可能情况的技术。我们做的这些逻辑学家称之为模型。

### 10.2 命题逻辑

命题逻辑使我们能只表示语言结构的对应与句子的特定连接词的那些部分。刚才我们看
了`and`。其他的连接词还有 `not`、`or` 和 `if...，then...`。命题逻辑形式中，这些连接词的对应形式有时叫做**布尔运算符**。命题逻辑的基本表达式是命题符号，通常写作 `P`、`Q`、`R` 等。表示布尔运算符的约定很多。由于我们将重点探索NLTK 中的逻辑表示方式，所以将使用下列 ASCII 版本的运算符：

In [1]:
import nltk
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


![list10-2](./imgs/list10-2.jpg)

NLTK 的`LogicParser()`将逻辑表达式分析成表达式的各种子类：

In [9]:
# lp = nltk.LogicParser()
lp = nltk.sem.Expression.fromstring('-(P & Q)')
# LogicParser.parse('-(P & Q)')
lp

<NegatedExpression -(P & Q)>

从假设一步一步推到结论，被称为推理。

回想一下，我们解释相对于一个模型的一种逻辑语言的句子，它们是这个世界的一个非 常简化的版本。一个命题逻辑的模型需要为每个可能的公式分配值`True`或 `False`。我们一步步的来做这个：首先，为每个命题符号分配一个值，然后确定布尔运算符的含义（即表10-2）和运用它们到这些公式的组件的值，来计算复杂的公式的值。估值是从逻辑的基本符号映射到它们的值。下面是一个例子：

我们使用一个配对的链表初始化一个估值，每个配对由一个语义符号和一个语义值组 成。所产生的对象基本上只是一个字典，映射逻辑符号（作为字符串处理）为适当的值。

In [10]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])

In [11]:
val['P']

True

In [12]:
dom = set([])
g = nltk.Assignment(dom)

In [13]:
m = nltk.Model(dom, val)

每一个模型都有一个`evaluate()`方法，可以确定逻辑表达式，如命题逻辑的公式，的 语义值；当然，这些值取决于最初我们分配给命题符号如`P`、`Q`和 `R`的真值。

In [14]:
print(m.evaluate('(P & Q)', g))

True


In [16]:
print(m.evaluate('-(P & Q)', g))

False


In [18]:
print(m.evaluate('(P & R)', g))

False


In [19]:
print(m.evaluate('(P | R)', g))

True


### 10.3 一阶逻辑

本章的剩余部分，我们将通过翻译自然语言表达式为一阶逻辑来表示它们的意思。并不 是所有的自然语言语义都可以用一阶逻辑表示。但它是计算语义的一个不错的选择，因为它具有足够的表现力来表达语义的很多方面，而且另一方面，有出色的现成系统可用于开展一阶逻辑自动推理。

下一步我们将描述如何构造一阶逻辑公式，然后是这样的公式如何用来评估模型。

#### 句法

一阶逻辑保留所有命题逻辑的布尔运算符，但它增加了一些重要的新机制。首先，命题 被分析成**谓词和参数**，这将我们与自然语言的结构的距离拉近了一步。一阶逻辑的标准构造规则承认以下术语：独立变量和独立常量、带不同数量的**参数**的**谓词**。例如：`Angus walks` 可以被形式化为`walk(angus)`，`Angus sees Bertie` 可以被形式化为 `see(angus, bertie)`。我们称 `walk` 为一元谓词，`see`为二元谓词。作为谓词使用的符号不具有内在的含义，虽然很难记住这一点。回到我们前面的一个例子，（13a）和（13b）之间没有逻辑区别。
`
(13) a. love(margrietje, brunoke) b. houden_van(margrietje, brunoke)
`

一阶逻辑本身没有什么实质性的关于词汇语义的表示——单个词的意思——虽然一些词汇语义理论可以用一阶逻辑编码。原子谓词如`see(angus, bertie)`在情况 `s` 中是真还是假不是一个逻辑的问题，而是依赖于特定的估值，即我们为常量`see`、`angus`和 `bertie`选择的值。 出于这个原因，这些表达式被称为**非逻辑常量**。相比之下，**逻辑常量**（如布尔运算符）在一阶逻辑的每个模型中的解释总是相同的。

我们应该在这里提到：有一个二元谓词具有特殊的地位，它就是等号，如在`angus=aj`这样的公式中的等号。等号被视为一个逻辑常量，因为对于单独的术语 `t1`和 `t2`，公式 `t1 = t2`为真当且仅当 `t1`和 `t2`是指同一个实体。

检查一阶逻辑表达式的语法结构往往是有益的，这样做通常的方式是为表达式指定类
型。下面是 `Montague` 文法的约定，我们将使用基本类型：`e`是实体类型，而 `t` 是公式类型，即有真值的表达式的类型。给定这两种基本类型，我们可以形成函数表达式的复杂类型。也就是说，给定任何类型`σ`和`τ`，`<σ,τ>`是一个对应与从`'σ things’`到`'τ things’`的函数的复杂类型。例如：`<e, T>`是从实体到真值，即一元谓词，的表达式的类型。可以调用`LogicParser`来进行类型检查。

In [37]:
import nltk
from nltk.sem.logic import LogicParser
tlp = LogicParser(type_check=True)
parsed = tlp.parse('walk(angus)')

In [38]:
parsed.argument

<ConstantExpression angus>

In [39]:
parsed.argument.type

e

In [40]:
parsed.function

<ConstantExpression walk>

In [41]:
parsed.function.type

<e,?>

In [42]:
# 要帮助类型检查器，我们需要指定一个信号，
# 作为一个字典来实施，明确的与非逻辑常量类型关联：
sig = {'walk': '<e, t>'}
parsed = tlp.parse('walk(angus)', sig)
parsed.function.type

<e,t>

In [44]:
lp = LogicParser()
lp.parse('dog(cyril)').free()

set()

#### 一阶定理证明
#### 一阶逻辑语言总结

我们将借此机会重新表述前面的命题逻辑的语法规则，并添加量词的形式化规则；所有 这些一起组成一阶逻辑的句法。此外，我们会明确相关表达式的类型。我们将采取约定：`<en, t>`是一种由 `n`个类型为`e` 的参数组成产生一个类型为 `t` 的表达式的谓词的类型。在这种情况下，我们说`n` 是谓词的元数。

1. 如果 `P` 是类型`<en, t>`的谓词，`α1，... αn`是 `e` 类型的术语，那么`P（α1，... αn）`的类型是`t`。
2. 如果`α`和`β`都是 `e` 类型，那么`（α=β）`和`（α!=β）`是 `t` 类型。 
3. 如果`φ`是 `t` 类型，那么是`-φ`也是 `t` 类型。 
4. 如果`φ`和`ψ`是 `t` 类型，那么`（φ&ψ）`、`（φ|ψ）`、`（φ -> ψ）`和`（φ<->ψ）`也是 `t` 类型。
5. 如果`φ`是 `t` 类型，`x`是类型为 `e` 的变量，那么`exists x.φ`和 `all x.φ`也是 `t` 类型。

#### 真值模型

In [9]:
dom = set(['b', 'o', 'c'])
v = """
bertie => b
olive => o 
cyril => c 
boy => {b}
girl => {o} 
dog => {c} 
walk => {o, c} 
see => {(b, o), (c, b), (o, c)} 
"""

#### 独立变量和赋值
#### 量化
#### 量词范围歧义
#### 模型的建立
### 10.4 英语句子的语义

#### 基于特征的文法中的合成语义学

组合原则：整体的含义是部分的含义与它们的句法结合方式的函数
#### `λ`演算

In [17]:
from nltk.sem.logic import LogicParser
lp = LogicParser()
e = lp.parse(r'\x.(walk(x) & chew_gum(x))')
e

<LambdaExpression \x.(walk(x) & chew_gum(x))>

In [19]:
e.free()

set()

In [20]:
print(lp.parse(r'\x.(walk(x) & chew_gum(y))'))

\x.(walk(x) & chew_gum(y))


In [21]:
e = lp.parse(r'\x.(walk(x) & chew_gum(x))(gerald)')
print(e)

\x.(walk(x) & chew_gum(x))(gerald)


In [22]:
print(e.simplify())

(walk(gerald) & chew_gum(gerald))


#### 量化的 `NP`
#### 及物动词
#### 再述量词歧义
### 10.5 段落语义层
段落是句子的序列。很多时候，段落中的一个句子的解释依赖它前面的句子。

#### 段落表示理论

段落表示理论（`Discourse Representation Theory，DRT`）的目标是提供一种方法处理这个和看上去是段落的特征的其它语义现象。一个段落表示结构（`discourse representation structure，DRS`）根据一个段落指称的列表和一个条件列表表示段落的意思。段落指称是段落中正在讨论的事情，它对应一阶逻辑的单个变量。`DRS`条件应用于那些段落指称，对应于一阶逻辑的原子开放公式。 

In [23]:
dp = nltk.DrtExpression.fromstring('([x, y], [angus(x), dog(y), own(x, y)])')
print(dp)

([x,y],[angus(x), dog(y), own(x,y)])


In [24]:
dp.draw()

In [26]:
print(dp.fol())

exists x y.(angus(x) & dog(y) & own(x,y))


### 10.6 小结

- 一阶逻辑是一种适合在计算环境中表示自然语言的含义的语言，因为它很灵活，足以表示自然语言含义的很多有用的方面，具有使用一阶逻辑推理的高效的定理证明器。（同样的，自然语言语义中也有各种各样的现象，需要更强大的逻辑机制。）
- 在将自然语言句子翻译成一阶逻辑的同时，我们可以通过检查一阶公式模型表述这些句子的真值条件。
- 为了构建成分组合的意思表示，我们为一阶逻辑补充了λ-演算。
- λ-演算中的β-约简在语义上与函数传递参数对应。句法上，它包括将被函数表达式中的λ绑定的变量替换为函数应用中表达式提供的参数。
- 构建模型的一个关键部分在于建立估值，为非逻辑常量分配解释。这些被解释为n 元谓词或独立常量。
- 一个开放表达式是一个包含一个或多个自变量的表达式。开放表达式只在它的自变量被赋值时被解释。
- 量词的解释是对于具有变量 `x` 的公式`φ[x]`，构建个体的集合，赋值 `g`分配它们作为 `x` 的值使`φ[x]`为真。然后量词对这个集合加以约束。
- 一个封闭的表达式是一个没有自由变量的表达式。也就是，变量都被绑定。一个封闭的表达式是真是假取决于所有变量赋值。
- 如果两个公式只是由绑定操作符（即λ或量词）绑定的变量的标签不同，那么它们是α-等价。重新标记公式中的绑定变量的结果被称为α-转换。
- 给定有两个嵌套量词 `Q1`和`Q2` 的公式，最外层的量词 `Q1`有较广的范围（或范围超出 `Q2`）。英语句子往往由于它们包含的量词的范围而产生歧义。
- 在基于特征的文法中英语句子可以通过将`SEM`作为特征与语义表达关联。一个复杂的表达式的`SEM`值通常包括成分表达式的 `SEM`值的函数应用。